In [1]:
from soccer_analytics.data.statsbomb import get_metadata, get_events
from soccer_analytics.data_split import split_by_time

In [2]:
competitions = get_metadata()

In [3]:
full_competitions = {
    "1. Bundesliga": ["2015/2016"],
#     "FA Women's Super League": ["2018/2019", "2019/2020", "2020/2021"],
#     "FIFA World Cup": ["2018", "2022"],
#     "Indian Super league": ["2021/2022"],
#     "La Liga": ["2015/2016"],
#     "Ligue 1": ["2015/2016"],
#     "Premier League": ["2015/2016"],
#     "Serie A": ["2015/2016"],
#     "UEFA Euro": ["2020"],
#     "UEFA Women's Euro": ["2022"],
#     "Women's World Cup": ["2019", "2023"]
}

In [32]:
from kloppy import statsbomb
test = statsbomb.load(
    "/Users/Rook/repos/soccer_analytics/data/statsbomb/events/3901254.json", 
    "/Users/Rook/repos/soccer_analytics/data/statsbomb/lineups/3901254.json",
    event_types=["shot"], coordinates="statsbomb",
)

In [33]:
type(test)

kloppy.domain.models.event.EventDataset

In [35]:
woo = []
woo.extend(test)
type(woo)

list

In [38]:
from kloppy.domain.models.event import EventDataset
EventDataset()

TypeError: EventDataset.__init__() missing 2 required positional arguments: 'records' and 'metadata'

In [4]:
shots = {}
for competition in competitions:
    print(competition.name, len(competition.seasons))
    if competition.name in full_competitions:
        shots[competition.name] = []
        for season in competition.seasons:
            print(f"    {season.name}: {len(season.matches)}")
            if season.name in full_competitions[competition.name]:
                shots[competition.name].extend(get_events(
                    season, event_types=["shot"]
                ))

1. Bundesliga 1
    2015/2016: 306
Champions League 18
Copa del Rey 3
FA Women's Super League 3
FIFA U20 World Cup 1
FIFA World Cup 8
Indian Super league 1
La Liga 18
Liga Profesional 2
Ligue 1 1
North American League 1
NWSL 1
Premier League 2
Serie A 2
UEFA Euro 1
UEFA Europa League 1
UEFA Women's Euro 1
Women's World Cup 2


In [5]:
all_shots = shots["1. Bundesliga"]

In [6]:
len(all_shots)

7831

In [ ]:
all_shots.

In [7]:
test = all_shots[0]

In [8]:
test

<CustomStatsBombShotEvent event_id='ba46e9d6-e828-4599-952c-39c1f7d22659' time='P1T04:21' player='Tarik Elyounoussi' result='OFF_TARGET'>

In [9]:
dir(test)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'absolute_timestamp',
 'ball_owning_team',
 'ball_state',
 'coordinates',
 'dataset',
 'event_id',
 'event_name',
 'event_type',
 'freeze_frame',
 'get_qualifier_value',
 'get_qualifier_values',
 'get_related_event',
 'get_related_events',
 'is_penalty',
 'matches',
 'next',
 'next_record',
 'period',
 'player',
 'prev',
 'prev_record',
 'qualifiers',
 'raw_event',
 'record_id',
 'related_ball_out',
 'related_card',
 'related_carry',
 'related_event_ids',
 'related_formation_change',

In [29]:
[shot.qualifiers for shot in all_shots]

[[BodyPartQualifier(value=<BodyPart.LEFT_FOOT: 'LEFT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.RIGHT_FOOT: 'RIGHT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.LEFT_FOOT: 'LEFT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.LEFT_FOOT: 'LEFT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.RIGHT_FOOT: 'RIGHT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.HEAD: 'HEAD'>)],
 [BodyPartQualifier(value=<BodyPart.HEAD: 'HEAD'>)],
 [BodyPartQualifier(value=<BodyPart.RIGHT_FOOT: 'RIGHT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.RIGHT_FOOT: 'RIGHT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.LEFT_FOOT: 'LEFT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.HEAD: 'HEAD'>)],
 [BodyPartQualifier(value=<BodyPart.LEFT_FOOT: 'LEFT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.RIGHT_FOOT: 'RIGHT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.LEFT_FOOT: 'LEFT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.LEFT_FOOT: 'LEFT_FOOT'>)],
 [BodyPartQualifier(value=<BodyPart.LEFT_FOOT: 'LEFT_FOOT'>)],
 [BodyPartQu

In [12]:
test.coordinates

Point(x=108.05, y=31.15)

In [23]:
[thing for _, thing in test.freeze_frame.players_coordinates.items()]

[Point(x=115.65, y=49.95),
 Point(x=113.15, y=39.25),
 Point(x=118.95, y=42.45),
 Point(x=111.55, y=34.75),
 Point(x=96.65, y=36.25),
 Point(x=103.95, y=43.75),
 Point(x=109.15, y=37.650000000000006),
 Point(x=111.55, y=41.85),
 Point(x=118.95, y=38.650000000000006),
 Point(x=93.75, y=39.650000000000006),
 Point(x=89.75, y=27.349999999999998),
 Point(x=103.45, y=50.35),
 Point(x=108.05, y=31.15)]

In [31]:
test.dataset.metadata.orientation

action-executing-team